In [1]:
# %matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import os
# Create a MirroredStrateg, If Multi-GPU available
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' 
tf.config.set_soft_device_placement(True)
# strategy = tf.distribute.MirroredStrategy(devices=['/gpu:1','/gpu:2']) 
# print('Number of GPUs being used: {}'.format(strategy.num_replicas_in_sync))
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model, model_from_json
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from livelossplot.keras import PlotLossesCallback
from livelossplot import PlotLossesKerasTF
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tqdm import tqdm
import efficientnet.keras as efn
import h5py, glob, re, cv2, math, matplotlib, pickle, gc
import tensorflow.keras as keras
import pandas as pd
from pandas import read_csv
import numpy as np
from scipy import stats
import itertools, random
from collections import Counter
# from cnn_utils import *
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
from PIL import Image  
import pdb
from statistics import mode 
from IPython.display import clear_output

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

np.random.seed(1)
# Setting the seed for python random numbers
random.seed(1254)
# Setting the graph-level random seed.
tf.random.set_seed(89)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
!CUDA_VISIBLE_DEVICES=2

# strategy = tf.distribute.MirroredStrategy(devices=['/gpu:1','/gpu:2'])
# print('Number of GPUs being used: {}'.format(strategy.num_replicas_in_sync))
# print('Number of GPUs being used: {}'.format(strategy.num_replicas_in_sync))
# def setup_multi_node_training(): # IMPORTANT: SET UP TF_CONFIG FOR MULTINODE TRAINING HERE os.environ[“TF_FORCE_GPU_ALLOW_GROWTH”] = “true” tf.config.set_soft_device_placement(True) mirrored_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(tf.distribute.experimental.CollectiveCommunication.NCCL) # Constructs the configuration run_config = tf.estimator.RunConfig( train_distribute=mirrored_strategy, ) return run_config
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Using TensorFlow backend.


Num GPUs Available:  1


## Load data

In [2]:
# filename = 'Datasets/final_imit_spec.pkl'
# filename = 'Datasets/final_imit_spec2_128.pkl'
filename = 'Datasets/final_imit_spec2_128_last.pkl'
with open(filename, 'rb') as input:
    x = pickle.load(input)
imit_train_images, imit_test_images, imit_y_train, imit_y_test, imit_env_train, imit_env_test = [x[0], x[1]
                                                                                                , x[2], x[3]
                                                                                                , x[4], x[5]]
print(imit_train_images.shape)
print(imit_test_images.shape)
print(imit_y_train.shape)
print(imit_y_test.shape)
print(imit_env_train.shape)
print(imit_env_test.shape)

(800, 120, 128, 128, 3)
(200, 120, 128, 128, 3)
(800, 120, 19)
(200, 120, 19)
(800,)
(200,)


In [3]:
filename = 'Datasets/final_imit_RD-RA2_128_last.hdf5'
data = h5py.File(filename, "r")
print('Selected File: '+str(filename))
imit_RD_train = np.array(data["train_RD"])
imit_RA_train = np.array(data["train_RA"])
imit_RD_test = np.array(data["test_RD"])
imit_RA_test = np.array(data["test_RA"])
data.close()
print(imit_RD_train.shape)
print(imit_RA_train.shape)
print(imit_RD_test.shape)
print(imit_RA_test.shape)

Selected File: Datasets/final_imit_RD-RA2_128_last.hdf5
(800, 605, 128, 128, 3)
(800, 605, 128, 128, 3)
(200, 605, 128, 128, 3)
(200, 605, 128, 128, 3)


In [4]:
# windowed RD
interval = range(0,600)
x_train11 = np.reshape(imit_RD_train[:,interval,:,:,:], (imit_RD_train.shape[0],120,5,imit_RD_train.shape[2],imit_RD_train.shape[3],imit_RD_train.shape[4]))
x_test11 = np.reshape(imit_RD_test[:,interval,:,:,:], (imit_RD_test.shape[0],120,5,imit_RD_test.shape[2],imit_RD_test.shape[3],imit_RD_test.shape[4]))
# y_train11 = np.reshape(imit_y_train_RD[:,interval], (imit_y_train_RD.shape[0],120,5))
# y_test11 = np.reshape(imit_y_test_RD[:,interval], (imit_y_test_RD.shape[0],120,5))
# y_train11 = to_categorical(np.squeeze(stats.mode(y_train11,2)[0]))
# y_test11 = to_categorical(np.squeeze(stats.mode(y_test11,2)[0]))
print(x_train11.shape)
# print(y_train11.shape)
print(x_test11.shape)
# print(y_test11.shape)  

# windowed RA
x_train12 = np.reshape(imit_RA_train[:,interval,:,:,:], (imit_RA_train.shape[0],120,5,imit_RA_train.shape[2],imit_RA_train.shape[3],imit_RA_train.shape[4]))
x_test12 = np.reshape(imit_RA_test[:,interval,:,:,:], (imit_RA_test.shape[0],120,5,imit_RA_test.shape[2],imit_RA_test.shape[3],imit_RA_test.shape[4]))
print(x_train12.shape)
print(x_test12.shape)

(800, 120, 5, 128, 128, 3)
(200, 120, 5, 128, 128, 3)
(800, 120, 5, 128, 128, 3)
(200, 120, 5, 128, 128, 3)


In [5]:
def data_generator(data, labels, batch_size=1):              
    """
    Yields the next training batch.
    data is an array  [[[frame1_filename,frame2_filename,…frame16_filename],label1], [[frame1_filename,frame2_filename,…frame16_filename],label2],……….].
    """
    num_samples = data.shape[0]
    
    while True:   
        for offset in range(0, num_samples, batch_size):
    #             print ('starting index: ', offset) 
            # Get the samples you'll use in this batch
            batch_samples = data[offset:offset+batch_size]
            label = labels[offset:offset+batch_size]
            # Initialise X_train and y_train arrays for this batch
            X_train = []
            y_train = []
            # For each example
            for i in range(0,batch_samples.shape[0]):
                X_train.append(batch_samples[i])
                y_train.append(label[i])

            # Make sure they're numpy arrays (as opposed to lists)
            X_train = np.array(X_train)
            #X_train = np.rollaxis(X_train,1,4)
            y_train = np.array(y_train)

            # yield the next training batch            
            yield X_train, y_train

In [6]:
def tdcnn2d_bilstm_seq():
    with tf.device('/gpu:2'):
#     with strategy.scope():
        inputlayer1 = Input(shape = (None,128,128,3))
        x = TimeDistributed(Conv2D(8, kernel_size=(3, 3), padding='same', activation='relu'))(inputlayer1)
        x = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(x)
#         x = BatchNormalization()(x)
        x = TimeDistributed(Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu'))(x)
        x = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(x)
#         x = BatchNormalization()(x)
        x = TimeDistributed(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))(x)
        x = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(x)
#         x = BatchNormalization()(x)
        x = TimeDistributed(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))(x)
        x = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(x)
#         x = BatchNormalization()(x)
        x = TimeDistributed(Flatten())(x)

#         x = TimeDistributed(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))(inputlayer1)
#         x = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(x)
# #         x = BatchNormalization()(x)
#         x = TimeDistributed(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))(x)
#         x = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(x)
# #         x = BatchNormalization()(x)
#         x = TimeDistributed(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))(x)
#         x = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(x)
#         x = TimeDistributed(MaxPooling2D(pool_size=(4,1)))(x)
#         x = TimeDistributed(Flatten())(x)

#         x = Reshape(target_shape = (-1,64*8*8))(x)
#         x = Dropout(0.5)(x)
#         x = Bidirectional(LSTM(8, dropout=0.5, return_sequences=True))(x)
        x = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True))(x)
#         x = Dropout(0.5)(x)
        out1 = Dense(num_class,activation='softmax')(x)
#         out1 = TimeDistributed(Dense(num_class,activation='softmax'))(x, mask=maskinput)
#         model = keras.Model(inputs = [inputlayer1,maskinput], outputs = [out1])#, out2, out3])
        model = keras.Model(inputs = [inputlayer1], outputs = [out1])#, out2, out3])
        opt = Adam(lr=1e-3, decay=1e-3 / 200)
        model.compile(loss = 'categorical_crossentropy', optimizer=opt,metrics = ['accuracy'])
        return model 

In [9]:
def tdcnn3d_bilstm_seq():
    with tf.device('/gpu:2'):
#     with strategy.scope():
        inputlayer1 = Input(shape = (None,5,128,128,3))
        x = TimeDistributed(Conv3D(8, kernel_size=(3, 3, 3), padding='same', activation='relu'))(inputlayer1)
        x = TimeDistributed(MaxPooling3D(pool_size=(1, 2, 2)))(x)
        x = TimeDistributed(Conv3D(16, kernel_size=(3, 3, 3), padding='same', activation='relu'))(x)
        x = TimeDistributed(MaxPooling3D(pool_size=(1, 2, 2)))(x)
        x = TimeDistributed(Conv3D(32, kernel_size=(3, 3, 3), padding='same', activation='relu'))(x)
        x = TimeDistributed(MaxPooling3D(pool_size=(1, 2, 2)))(x)
        x = TimeDistributed(Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu'))(x)
        x = TimeDistributed(MaxPooling3D(pool_size=(1, 2, 2)))(x)
        x = TimeDistributed(MaxPooling3D(pool_size=(1, 4, 1)))(x)
        x = TimeDistributed(Flatten())(x)
#         x = Reshape(target_shape = (-1, 64*8*8*5))(x)
        x = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True))(x)
        
        out1 = Dense(num_class,activation='softmax')(x)
#         out1 = TimeDistributed(Dense(num_class,activation='softmax'))(x, mask=maskinput)
#         model = keras.Model(inputs = [inputlayer1,maskinput], outputs = [out1])#, out2, out3])
        model = keras.Model(inputs = [inputlayer1], outputs = [out1])#, out2, out3])
        opt = Adam(lr=1e-3, decay=1e-3 / 200)
        model.compile(loss = 'categorical_crossentropy', optimizer=opt,metrics = ['accuracy'])
        return model 

In [10]:
early_stopping_patience = 10
# Add early stopping
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=early_stopping_patience, restore_best_weights=True
)

In [15]:
seqlen = [1, 2, 3, 6, 12, 24]
num_class = imit_y_train.shape[-1]
for i in range(len(seqlen)):
    xtr_md = imit_train_images.reshape(-1,120//seqlen[i],128,128,3)
    xts_md = imit_test_images.reshape(-1,120//seqlen[i],128,128,3)
    ytr = imit_y_train.reshape(-1,120//seqlen[i],19)
    yts = imit_y_test.reshape(-1,120//seqlen[i],19)
    xtr_rd = x_train11.reshape(-1,120//seqlen[i],5,128,128,3)
    xts_rd = x_test11.reshape(-1,120//seqlen[i],5,128,128,3)
    xtr_ra = x_train12.reshape(-1,120//seqlen[i],5,128,128,3)
    xts_ra = x_test12.reshape(-1,120//seqlen[i],5,128,128,3)
    print('Training '+str(seqlen[i])+' sec models...')
    
    # md
    model = tdcnn2d_bilstm_seq()
    train_dataset = data_generator(xtr_md, ytr)
    validation_dataset = data_generator(xts_md, yts)
    history = model.fit(
        train_dataset,
        validation_data=validation_dataset,
        batch_size = 32,
        epochs=100,
        steps_per_epoch = len(ytr), validation_steps = len(yts),
        callbacks=[early_stopping],
        verbose = 0
    )
    loss, acc = model.evaluate(validation_dataset, steps = len(yts))
    print('md acc: '+str(acc))
    fname = 'final varying md len_'+str(seqlen[i])
    MODEL_FILE = 'Models/' + fname + '.json' # save path
    WEIGHT_FILE = 'Models/' + fname + '.h5' # save path

    model_json = model.to_json()
    with open(MODEL_FILE, "w") as json_file:
        json_file.write(model_json)
    model.save_weights(WEIGHT_FILE)
    
    # rd
    model = tdcnn3d_bilstm_seq()
    train_dataset = data_generator(xtr_rd, ytr)
    validation_dataset = data_generator(xts_rd, yts)
    history = model.fit(
        train_dataset,
        validation_data=validation_dataset,
        batch_size = 32,
        epochs=100,
        steps_per_epoch = len(ytr), validation_steps = len(yts),
        callbacks=[early_stopping],
        verbose = 0
    )
    loss, acc = model.evaluate(validation_dataset, steps = len(yts))
    print('rd acc: '+str(acc))
    fname = 'final varying rd len_'+str(seqlen[i])
    MODEL_FILE = 'Models/' + fname + '.json' # save path
    WEIGHT_FILE = 'Models/' + fname + '.h5' # save path

    model_json = model.to_json()
    with open(MODEL_FILE, "w") as json_file:
        json_file.write(model_json)
    model.save_weights(WEIGHT_FILE)
    
    # ra
    model = tdcnn3d_bilstm_seq()
    train_dataset = data_generator(xtr_ra, ytr)
    validation_dataset = data_generator(xts_ra, yts)
    history = model.fit(
        train_dataset,
        validation_data=validation_dataset,
        batch_size = 32,
        epochs=100,
        steps_per_epoch = len(ytr), validation_steps = len(yts),
        callbacks=[early_stopping],
        verbose = 0
    )
    loss, acc = model.evaluate(validation_dataset, steps = len(yts))
    print('ra acc: '+str(acc))
    fname = 'final varying ra len_'+str(seqlen[i])
    MODEL_FILE = 'Models/' + fname + '.json' # save path
    WEIGHT_FILE = 'Models/' + fname + '.h5' # save path

    model_json = model.to_json()
    with open(MODEL_FILE, "w") as json_file:
        json_file.write(model_json)
    model.save_weights(WEIGHT_FILE)

Training 1 sec models...
200/200 [==============================] - 7s 37ms/step - loss: 0.6415 - accuracy: 0.8605
md acc: 0.8604999780654907
200/200 [==============================] - 9s 46ms/step - loss: 0.2231 - accuracy: 0.9240
rd acc: 0.9239583611488342
200/200 [==============================] - 9s 47ms/step - loss: 0.3501 - accuracy: 0.8972
ra acc: 0.8971666693687439
Training 2 sec models...
400/400 [==============================] - 7s 17ms/step - loss: 0.8797 - accuracy: 0.8431
md acc: 0.8430833220481873
400/400 [==============================] - 10s 24ms/step - loss: 0.4789 - accuracy: 0.9003
rd acc: 0.9002916812896729
400/400 [==============================] - 10s 25ms/step - loss: 0.3999 - accuracy: 0.8593
ra acc: 0.859333336353302
Training 3 sec models...
600/600 [==============================] - 6s 11ms/step - loss: 0.6568 - accuracy: 0.8456
md acc: 0.8455833196640015
600/600 [==============================] - 10s 17ms/step - loss: 0.5064 - accuracy: 0.8991
rd acc: 0.8991

# sec prints are reverse !

In [7]:
def loadmodel(fname):
    model_file = 'Models/' + fname + '.json'
    w_file = 'Models/' + fname + '.h5'
    json_file = open(model_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model_md = model_from_json(loaded_model_json)
    model_md.load_weights(w_file)
    return model_md

In [8]:
seqlen = [1, 2, 3, 6, 12, 24]
mod_md_1sec = loadmodel('final varying md len_'+str(seqlen[-1]))
mod_md_2sec = loadmodel('final varying md len_'+str(seqlen[-2]))
mod_rd_1sec = loadmodel('final varying rd len_'+str(seqlen[-1]))
mod_rd_2sec = loadmodel('final varying rd len_'+str(seqlen[-2]))
mod_ra_1sec = loadmodel('final varying ra len_'+str(seqlen[-1]))
mod_ra_2sec = loadmodel('final varying ra len_'+str(seqlen[-2]))

In [9]:
models = [mod_md_1sec,mod_md_2sec,mod_rd_1sec,mod_rd_2sec,mod_ra_1sec,mod_ra_2sec]

In [10]:
import time

In [ ]:
xts_md = imit_test_images.reshape(-1,120//seqlen[-1],128,128,3)
t = time.time()
p = mod_md_1sec.predict(xts_md,batch_size=1,verbose=0)
time.time() - t

In [20]:
xts_md = imit_test_images.reshape(-1,120//seqlen[-2],128,128,3)
t = time.time()
p = mod_md_2sec.predict(xts_md,batch_size=512,verbose=0)
time.time() - t

10.318246603012085

In [15]:
xts_rd = x_test11.reshape(-1,120//seqlen[-1],5,128,128,3)
t = time.time()
p = mod_rd_1sec.predict(xts_rd,batch_size=128,verbose=0)
time.time() - t

14.793370246887207

In [16]:
xts_rd = x_test11.reshape(-1,120//seqlen[-2],5,128,128,3)
t = time.time()
p = mod_rd_2sec.predict(xts_rd,batch_size=128,verbose=0)
time.time() - t

14.334917783737183

In [17]:
xts_ra = x_test12.reshape(-1,120//seqlen[-1],5,128,128,3)
t = time.time()
p = mod_ra_1sec.predict(xts_ra,batch_size=128,verbose=0)
time.time() - t

15.35310673713684

In [18]:
xts_ra = x_test12.reshape(-1,120//seqlen[-2],5,128,128,3)
t = time.time()
p = mod_ra_2sec.predict(xts_ra,batch_size=128,verbose=0)
time.time() - t

14.054130792617798